In [11]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from HTML import preprocessing as ps
%matplotlib inline
data = pd.read_csv(Config.train_path)
data = ps.preprocessing(data)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91527 entries, 0 to 91526
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              91527 non-null  int64  
 1   hotel                           91527 non-null  object 
 2   is_canceled                     91527 non-null  int64  
 3   lead_time                       91527 non-null  int64  
 4   arrival_date_year               91527 non-null  int64  
 5   arrival_date_month              91527 non-null  object 
 6   arrival_date_week_number        91527 non-null  int64  
 7   arrival_date_day_of_month       91527 non-null  int64  
 8   stays_in_weekend_nights         91527 non-null  int64  
 9   stays_in_week_nights            91527 non-null  int64  
 10  adults                          91527 non-null  int64  
 11  children                        91527 non-null  float64
 12  babies                          

In [12]:
data = data.drop(['reservation_status','reservation_status_date', 'adr'], axis=1)
data = pd.concat([data, 
                 pd.get_dummies(data['hotel'], drop_first=True), 
                 pd.get_dummies(data['arrival_date_month'], drop_first=True), 
                 pd.get_dummies(data['market_segment'], drop_first=True),
                 pd.get_dummies(data['distribution_channel'], drop_first=True),
                 pd.get_dummies(data['customer_type'], drop_first=True), 
                 pd.get_dummies(data['deposit_type'], drop_first=True),
                 pd.get_dummies(data['meal'], drop_first=True),
                 pd.get_dummies(data['country'], drop_first=True),
                 pd.get_dummies(data['reserved_room_type'], drop_first=True),
                 pd.get_dummies(data['assigned_room_type'], drop_first = True)
                 ], axis=1)
data = data.drop(['assigned_room_type', 'reserved_room_type', 'meal', 'country', 'hotel','arrival_date_month','market_segment','distribution_channel','customer_type', 'deposit_type'], axis=1)
data = data.drop(['ID', 'agent', 'company'], axis=1)
data.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,C,D,E,F,G,H,I,K,L,P
0,0,342,2015,27,1,0,0,2,0.0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,257,2015,27,1,0,2,1,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91527 entries, 0 to 91526
Columns: 231 entries, is_canceled to P
dtypes: float64(1), int64(16), uint8(214)
memory usage: 30.6 MB


In [14]:
from sklearn.model_selection import cross_val_score
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = np.unique(y_train)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights



In [18]:
# Fit and train
s = []
x = data[set(data.columns) - {'is_canceled', }]
y = data['is_canceled']
from sklearn.ensemble import RandomForestClassifier
for n in [1, 5, 10, 50, 100, 500]:
    clf = RandomForestClassifier(n_estimators=n, max_depth = 8, criterion='entropy', random_state=1126)
    scores = cross_val_score(clf, x, y, cv=5, scoring='f1', verbose=1, n_jobs=2)
    s.append(scores)
    print(np.mean(scores))
    
# 500 is the best

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    2.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


0.5610558250428945


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


0.5900771785679538


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    6.8s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


...

[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   25.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


0.5742234867870761


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   48.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


0.5793447601605602
0.5822056897864486


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:  3.4min finished


In [ ]:
plt.plot([1,5,10,50,100,500], np.mean(np.array(s),axis=1))

In [16]:
clf = RandomForestClassifier(n_estimators=1000, criterion='entropy', random_state=0)
scores = cross_val_score(clf, x, y, cv=5, scoring='f1', verbose=1, n_jobs=2)
s.append(scores)
print(np.mean(scores))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


0.7149952770120274


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:  9.2min finished
